# Project X

In [ ]:
import os

import re
import pandas as pd
import numpy as np

In [ ]:
cwd = os.getcwd()
df = pd.read_json(cwd + "/training_set.json")
df.data = pd.DataFrame(df.data.values.tolist())
#df.paragraphs = pd.DataFrame(df.data.values.tolist())['paragraphs']
#df = df.groupby(['date','name','number'])['locations'].apply(','.join).reset_index()
print(df)

                         data  version
0    University_of_Notre_Dame      1.1
1                     Beyoncé      1.1
2                     Montana      1.1
3                    Genocide      1.1
4                 Antibiotics      1.1
..                        ...      ...
437                    Police      1.1
438          Punjab,_Pakistan      1.1
439                 Infection      1.1
440                   Hunting      1.1
441                 Kathmandu      1.1

[442 rows x 2 columns]
